In [ ]:
pip install flask-ngrok

In [ ]:
from google.colab import auth
from google.colab import files
from google.colab import drive
from keras.preprocessing import image
import tensorflow as tf
import numpy as np
import base64
import sys
from flask import Flask, request
from flask_ngrok import run_with_ngrok
import os
import json
import tensorflow as tf
from PIL import Image
import io

# Flasking

In [ ]:
# Mount drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# from keras.preprocessing import image
model = tf.keras.models.load_model('/content/drive/MyDrive/Kaggle/models/mobilenet_v1/mobilenetv3_edit_v1')
class_names = ['freshapples', 'freshbanana', 'freshlemon', 'freshoranges',
               'rottenapples', 'rottenbanana', 'rottenlemon', 'rottenoranges']

def preproc(data):
    imgdata = base64.decodebytes(data)
    image = Image.open(io.BytesIO(imgdata)).convert('RGB')
    image_new = image.resize((224,224))
    x = np.array(image_new)
    x = np.expand_dims(x, axis=0)
    x = x / 255.0
    new_x = np.vstack([x])
    return new_x

def do_predict(data):
    prediction = model.predict(data)
    new_prediction = prediction.tolist()
    result = int(np.argmax(prediction))
    class_name = class_names[result]
    percent = float(np.max(prediction * 100))
    jsonDict = {
        'predClass': result,
        'className': class_name,
        'percentage': percent,
        'prediction': new_prediction,
    }
    return json.dumps(jsonDict)

In [ ]:
app = Flask(__name__)
run_with_ngrok(app)


@app.route('/', methods=['GET'])
def index():
    data = {'status': 200, 'data': 'Hello there'}
    return json.dumps(data)

@app.route('/api/predict/', methods=['POST'])
def predict_img():
    item = request.json
    item_dict = json.loads(item)
    jsonResponse = do_predict(item_dict['data'])
    return jsonResponse

@app.route('/api/test/', methods=['POST'])
def test_post():
    item = request.json
    jsonResult = {
        'data': item,
    }
    return json.dumps(jsonResult)

@app.route('/api/predict/img/', methods=['POST'])
def predict_full_img():
    item = request.json
    item_dict = json.loads(item, strict=False)
    encodedItem = item_dict['data'].encode('utf8')
    img_data = preproc(encodedItem)
    jsonResponse = do_predict(img_data)
    return jsonResponse

if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://08bc9f475119.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [31/May/2021 15:29:41] "POST /api/predict/ HTTP/1.1" 200 -
127.0.0.1 - - [31/May/2021 15:29:41] "POST /api/predict/img HTTP/1.1" 308 -
127.0.0.1 - - [31/May/2021 15:29:42] "POST /api/predict/img/ HTTP/1.1" 200 -
127.0.0.1 - - [31/May/2021 15:29:45] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [31/May/2021 15:30:59] "POST /api/predict/img/ HTTP/1.1" 200 -
127.0.0.1 - - [31/May/2021 15:34:35] "POST /api/predict/img/ HTTP/1.1" 200 -
127.0.0.1 - - [31/May/2021 15:36:07] "POST /api/predict/img/ HTTP/1.1" 200 -
127.0.0.1 - - [31/May/2021 15:37:31] "POST /api/predict/img/ HTTP/1.1" 200 -
